Our scenario continues:

As a customer analyst, I want to know who has spent the most money with us over their lifetime. I have monthly charges and tenure, so I think I will be able to use those two attributes as features to estimate total_charges. I need to do this within an average of $5.00 per customer.

create a notebook named model.ipynb.

#### run all your previous scripts that acquired, prepared, split, and scaled your data.

In [1]:
import wrangle
import split_scale
import features

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings("ignore")

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_sel

In [2]:
#acquire data
df = wrangle.wrangle_telco()
df.drop(columns = "customer_id", inplace=True)
df["dummy_col"] = 1
df.head()

,monthly_charges,tenure,total_charges,dummy_col
0,109.70,71,7904.25,1
1,84.65,63,5377.80,1
2,90.45,65,5957.90,1
3,45.20,54,2460.55,1
4,116.80,72,8456.75,1


In [37]:
#split data
train, test = split_scale.split_my_data(df,.80)
X_train = train.drop(columns="total_charges")
y_train = train["total_charges"]
X_test = test.drop(columns="total_charges")
y_test = test["total_charges"]

In [38]:
#scale X data only
X_train_scaled, X_test_scaled, scaler = split_scale.standard_scaler(X_train, X_test)

#### select your features using your features script

In [39]:
selected_features = features.select_kbest_freg_scaled(X_train_scaled, y_train, 2)

In [40]:
X_train, X_test = features.features_to_X_train_and_test(selected_features)

fit 2 different linear models to your data

In [41]:
lm1 = LinearRegression()

In [42]:
lm1.fit(X_train, y_train)
print("Linear Model:", lm1)

lm1_y_intercept = lm1.intercept_
print("intercept: ", lm1_y_intercept)

lm1_coefficients = lm1.coef_
print("coefficients: ", lm1_coefficients)

Linear Model: LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
intercept:  3740.53690652819
coefficients:  [2124.43276129  741.9671209 ]


In [43]:
print('{} = b + m1 * {} + m2 * {}'.format(y_train.name[0], X_train.columns[0],X_train.columns[1]))
print('    y-intercept  (b): %.2f' % lm1_y_intercept)
print('    coefficient (m1): %.2f' % lm1_coefficients[0])
print('    coefficient (m2): %.2f' % lm1_coefficients[1])

t = b + m1 * monthly_charges + m2 * tenure
    y-intercept  (b): 3740.54
    coefficient (m1): 2124.43
    coefficient (m2): 741.97


In [44]:
y_pred_lm1 = lm1.predict(X_train)

In [45]:
mse_lm1 = mean_squared_error(y_train, y_pred_lm1)
print("linear model\n  mean squared error: {:.3}".format(mse_lm1)) 

r2_lm1 = r2_score(y_train, y_pred_lm1)
print('  {:.2%} of the variance in the student''s final grade can be explained by the grades on exam 1 and 3.'.format(r2_lm1))

NameError: name 'mean_squared_error' is not defined

evaluate the 2 models and your baseline

select the one that performed the best

apply to your test data

Write a function that creates the model object, fits and predicts, given X_train, X_test, y_train, y_test

Write a function, plot_residuals(x, y, dataframe) that takes the feature, the target, and the dataframe as input and returns a residual plot.

Write a function, plot_regression(x, y) that takes a feature and a target and returns the datapoints, the regression line, and the confidence interval. (only plotting a single feature with y) (hint: confidence interval - take advantage of things already written)